# Proyecto 1
Daniel Machic 22118 - Maria José

## Obtención de Datos

In [1]:
import pandas as pd
import os
import re
from IPython.display import display, FileLink
import unicodedata


def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[\x00-\x1F\x7F]', '', text)
        text = text.replace('"', "'").replace('“', "'").replace('”', "'")
    return text

def limpiar_filas_finales(df):
    """Elimina el patrón específico del final de los archivos"""
    # Identificar filas completamente vacías
    filas_vacias = df.isnull().all(axis=1)

    # Buscar el inicio del patrón final (2 vacías + texto + 1 vacía + copyright)
    inicio_pie = None
    for i in range(len(df)-4, -1, -1):  # Busca desde el final
        # Verifica el patrón: vacía, vacía, texto, vacía, copyright
        if (filas_vacias[i] and
            filas_vacias[i+1] and
            'Establecimientos encontrados' in str(df.iloc[i+2].values) and
            filas_vacias[i+3] and
            'Ministerio de Educaci' in str(df.iloc[i+4].values)):
            inicio_pie = i
            break

    # Si encontramos el patrón, cortamos el DataFrame
    if inicio_pie is not None:
        return df.iloc[:inicio_pie]
    return df

# Configuración principal
ruta_carpeta = "/Users/danielmachic/Desktop/Data Science/Proyecto1/Departamentos/"

if not os.path.exists(ruta_carpeta):
    raise FileNotFoundError(f"Carpeta no encontrada: {ruta_carpeta}")

archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.xls', '.xlsx'))]
if not archivos:
    raise ValueError("No se encontraron archivos Excel")

print(f"Procesando {len(archivos)} archivos...")

dataframes = []
for archivo in archivos:
    try:
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        engine = 'openpyxl' if archivo.lower().endswith('.xlsx') else 'xlrd'
        df = pd.read_excel(ruta_completa, skiprows=27, engine=engine)

        # Limpieza de filas finales
        df = limpiar_filas_finales(df)

        # Limpieza de datos
        df = df.rename(columns={'DEPARTAMENTO': 'Departamento'})
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].apply(clean_text)

        dataframes.append(df)
        print(f"✔ {archivo} procesado (filas conservadas: {len(df)})")

    except Exception as e:
        print(f"✖ Error en {archivo}: {str(e)}")
        continue

# Consolidación y guardado (igual que antes)
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

    print("\nResumen final:")
    print(f"- Total registros: {len(df)}")
    print(f"- Departamentos únicos: {df['Departamento'].nunique()}")
    print("\nMuestra de datos:")
    display(df.head(10))

    try:
        output_excel = os.path.join(ruta_carpeta, "../datos_consolidados.xlsx")
        df.to_excel(output_excel, index=False, engine='openpyxl')
        print(f"\n✅ Archivo guardado en: {output_excel}")
        display(FileLink(output_excel))
    except Exception as e:
        output_csv = os.path.join(ruta_carpeta, "../datos_consolidados.csv")
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        print(f"\n⚠️ Se guardó como CSV: {output_csv}")
        display(FileLink(output_csv))
else:
    print("❌ No se procesaron archivos válidos")

Procesando 23 archivos...
✔ El_Progreso.xls procesado (filas conservadas: 145)
✔ Zacapa.xls procesado (filas conservadas: 135)
✔ Chimaltenango.xls procesado (filas conservadas: 423)
✔ Retalhuleu.xls procesado (filas conservadas: 344)
✔ Ciudad_Capital.xls procesado (filas conservadas: 2122)
✔ Escuintla.xls procesado (filas conservadas: 698)
✔ Huehuetenango.xls procesado (filas conservadas: 557)
✔ Jutiapa.xls procesado (filas conservadas: 369)
✔ Jalapa.xls procesado (filas conservadas: 177)
✔ Totonicapan.xls procesado (filas conservadas: 109)
✔ Quiche.xls procesado (filas conservadas: 280)
✔ Guatemala.xls procesado (filas conservadas: 1813)
✔ Quetzaltenango.xls procesado (filas conservadas: 535)
✔ Suchitepequez.xls procesado (filas conservadas: 430)
✔ Santa_Rosa.xls procesado (filas conservadas: 194)
✔ Sacatepequez.xls procesado (filas conservadas: 408)
✔ Alta_Verapaz.xls procesado (filas conservadas: 424)
✔ Chiquimula.xls procesado (filas conservadas: 222)
✔ Solola.xls procesado (filas 

,CODIGO,Unnamed: 1,DISTRITO,Departamento,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,Unnamed: 18,Unnamed: 19
0,02-01-0027-46,NaN,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO TECNICO INDUSTRIAL MIXTO GUASTATOYA,BARRIO EL CALVARIO,79450881,CARLA MARLENY ALDANA RODAS,JOSɠARTURO LӐEZ ORTIZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),EL PROGRESO,NaN,NaN
1,02-01-0028-46,NaN,02-014,EL PROGRESO,GUASTATOYA,COLEGIO DE CIENCIAS COMERCIALES EL PROGRESO,BARRIO EL PORVENIR,79451265,CARLA MARLENY ALDANA RODAS,DANIEL LӐEZ SOL͓,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),EL PROGRESO,NaN,NaN
2,02-01-0031-46,NaN,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,BARRIO EL PORVENIR,54422753,CARLA MARLENY ALDANA RODAS,ILIANA ARACELY LZARO HERNNDEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO,NaN,NaN
3,02-01-0038-46,NaN,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO DE EDUCACION MEDIA POR MADUREZ,"COLONIA NUEVA VIDA, BARRIO EL PORVENIR",7945177,CARLA MARLENY ALDANA RODAS,NORA REBECA IBAхZ MORN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO,NaN,NaN
4,02-01-0044-46,NaN,02-004,EL PROGRESO,GUASTATOYA,INSTITUTO OFICIAL DE EDUCACION BASICA Y BACHIL...,BARRIO LAS JOYAS,56962830,EDGAR RUDY MORALES GODINEZ,MARIA ELENA LEMUS MURALLES DE AREVALO,DIVERSIFICADO,OFICIAL,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO,NaN,NaN
5,02-01-0045-46,NaN,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO DE EDUCACIӎ MEDIA POR MADUREZ,BARRIO EL PORVENIR,56719955,CARLA MARLENY ALDANA RODAS,NORA REBECA IBAхZ MORN,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO,NaN,NaN
6,02-01-0046-46,NaN,02-014,EL PROGRESO,GUASTATOYA,COLEGIO DE APLICACIONES INTEGRADAS GUASTATOYA,BARRIO EL PORVENIR,79450840,CARLA MARLENY ALDANA RODAS,BRENDA AZUCENA VALIENTE ARREAGA,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,SABATINO,EL PROGRESO,NaN,NaN
7,02-01-0049-46,NaN,02-021,EL PROGRESO,GUASTATOYA,CENTRO MUNICIPAL DE EDUCACIӎ EXTRAESCOLAR -CEEX-,COLONIA LINDA VISTA,58250827,MARTA ELIDA CARIAS HERNANDEZ DE GARCIA,SANDRA PAOLA MORALES GARĆ,DIVERSIFICADO,MUNICIPAL,RURAL,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO,NaN,NaN
8,02-01-0054-46,NaN,02-012,EL PROGRESO,GUASTATOYA,COLEGIO EVANGELICO TORRE FUERTE,"COLONIA HICHOS, BARRIO EL PORVENIR",79451993,CARLOTA EUGENIA ALBUREZ AGUILAR,IRIS ORFEL͠LOAIZA MOSCOSO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO,NaN,NaN
9,02-01-0057-46,NaN,02-014,EL PROGRESO,GUASTATOYA,COLEGIO HIGA,COLONIA HICHOS,53947190,CARLA MARLENY ALDANA RODAS,ERIKA JULISA OLIVA DUARTE,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO,NaN,NaN



✅ Archivo guardado en: /Users/danielmachic/Desktop/Data Science/Proyecto1/Departamentos/../datos_consolidados.xlsx


/Users/danielmachic/Desktop/Data Science/Proyecto1/datos_consolidados.xlsx

## Limpieza de Datos

#### Eliminación de columnas vacías

In [2]:
df = df.drop(columns=['Unnamed: 1', 'Unnamed: 18', 'Unnamed: 19'])
print(df.columns.tolist())  # Para ver los nombres exactos de las columnas

['CODIGO', 'DISTRITO', 'Departamento', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']


Se eliminaron 3 columnas ya que a la hora de hacer la unión de todos los archivos, se crearon 3 columnas sin información.

## Columna Distrito

In [3]:
#Valores vacíos en la columna Distrito
filas_vacias = df[df['DISTRITO'].isna()]
num_filas = len(filas_vacias)
print("Número de filas:", num_filas)

Número de filas: 525


No se eliminarán las filas con valores vacíos en la columna Distrito, ya que podrían afectar el análisis. Actualmente existen 523 valores vacíos, por lo que, en lugar de eliminarlos, se procederá a modificar la información, no solo en esa columna, sino también en otras.

In [4]:
df['DISTRITO'] = df['DISTRITO'].fillna('NO ESPECIFICADO')

In [5]:
conteo = (df['DISTRITO'] == '01-').sum()
print(f"El valor '01-' aparece {conteo} veces en la columna DISTRITO.")

# Reemplazar '01-' por 'NO ESPECIFICADO' en la columna DISTRITO
df['DISTRITO'] = df['DISTRITO'].replace('01-', 'NO ESPECIFICADO')


El valor '01-' aparece 66 veces en la columna DISTRITO.


El valor "01-" aparecía 66 veces y representaba un dato incompleto; por ello, se decidió reemplazarlo por "NO ESPECIFICADO", al igual que los valores vacíos, con el fin de no afectar el análisis.

## Columnas Departamento y MUNICIPIO

In [6]:
columnas = ['Departamento', 'MUNICIPIO']
df[columnas] = df[columnas].apply(lambda x: x.str.upper().str.replace(r'\s+', ' ', regex=True).str.strip())

Se asegura que las columnas tengan un mismo formato y se elimina cualquier espacio al inicio o al final

## Columna Establecimiento

In [7]:
df = df.dropna(subset=['ESTABLECIMIENTO'])

Se eliminan las columnas donde "Establecimiento" no tiene información ya que al revisar el campo de "Status", estas se encuentran cerradas definitivamente. Además en el campo "Distrito" estas no cuentan con información al igual que en el campo de "Director", estos siendo campos importantes en la base de datos

In [11]:
def limpiar_establecimiento(texto):
    if pd.isna(texto):
        return texto

    # Convertir a string
    texto = str(texto)

    # Eliminar caracteres invisibles (como \u200b, \ufeff, etc.)
    texto = re.sub(r"[\u200B-\u200D\uFEFF]", "", texto)

    # Mantener tildes y ñ — normalización NFKC preserva caracteres latinos con acentos
    texto = unicodedata.normalize("NFKC", texto)

    # Eliminar comillas simples y dobles
    texto = texto.replace("'", "").replace('"', "")

    # Eliminar guiones, paréntesis y otros signos de puntuación comunes que no aportan
    texto = re.sub(r"[-()]", " ", texto)

    # Eliminar cualquier otro símbolo que no sea letra, número o espacio (excepto tildes y ñ)
    texto = re.sub(r"[^A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9 ]", " ", texto)

    # Reducir múltiples espacios a uno solo
    texto = re.sub(r"\s+", " ", texto)

    # Quitar espacios al inicio y final
    texto = texto.strip()

    # Pasar a mayúsculas
    texto = texto.upper()

    return texto

# Aplicar limpieza
df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO"].apply(limpiar_establecimiento)

0           INSTITUTO TECNICO INDUSTRIAL MIXTO GUASTATOYA
1             COLEGIO DE CIENCIAS COMERCIALES EL PROGRESO
2           INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA
3                INSTITUTO DE EDUCACION MEDIA POR MADUREZ
4       INSTITUTO OFICIAL DE EDUCACION BASICA Y BACHIL...
                              ...                        
996     INSTITUTO MUNICIPAL DE EDUCACION MEDIA Y BACHI...
997      CENTRO EDUCATIVO MIXTO FUTURO DE LOS NI S CEFUNI
998                 COLEGIO MIXTO PAULO COELHO CHAMPERICO
999                 COLEGIO MIXTO PAULO COELHO CHAMPERICO
1000    INSTITUTO MUNICIPAL DE EDUCACI BSICA Y BACHILL...
Name: ESTABLECIMIENTO, Length: 1000, dtype: object


In [70]:

df['DIRECCION'] = df['DIRECCION'].fillna('SIN DIRECCIÓN')
df['SUPERVISOR'] = df['SUPERVISOR'].fillna('SIN DATO')
df['DIRECTOR'] = df['DIRECTOR'].fillna('SIN DATO')

En su lugar se cambiaron los valores vacíos por información

#### Caracteres especiales

In [54]:
num_filas = len(df)
print("Número de filas:", num_filas)

Número de filas: 11273


In [58]:
filas_vacias = df[df['DISTRITO'].isna()]
num_filas = len(filas_vacias)
print("Número de filas:", num_filas)


Número de filas: 523


,CODIGO,DISTRITO,Departamento,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
20,02-01-0230-46,NaN,EL PROGRESO,GUASTATOYA,CENTRO EDUCATIVO 'GUAYACAN',BARRIO EL GOLFO,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),EL PROGRESO
45,02-03-0102-46,NaN,EL PROGRESO,SAN AGUSTIN ACASAGUASTLAN,'LICEO SAN JOSE',BARRIO EL CENTRO ALDEA EL RANCHO,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,RURAL,TEMPORAL TITULOS,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
85,02-05-0132-46,NaN,EL PROGRESO,EL JICARO,ESCUELA PARTICULAR MIXTA DE CIENCIAS COMERCIALES,EL JICARO,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
86,02-05-0251-46,NaN,EL PROGRESO,EL JICARO,INSTITUTO DE EDUCACION BACHILLERATO POR MADUREZ,EL JICARO,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),EL PROGRESO
89,02-05-0592-46,NaN,EL PROGRESO,EL JICARO,COLEGIO DIVERSIFICADO EL JICARO,BARRIO BUENOS AIRES,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10978,12-14-0023-46,NaN,SAN MARCOS,SAN JOSE EL RODEO,COLEGIO TECNICO PARTICULAR 'CORAZON DE JESUS',4A. CALLE ZONA 4,40608399,NaN,RAڌ CRISPINIANO IXLAJ RAMIREZ,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,FIN DE SEMANA,SAN MARCOS
11072,12-15-4193-46,NaN,SAN MARCOS,MALACATAN,INSTITUTO NORMAL MIXTO 'MALACATAN',NaN,NaN,NaN,NaN,DIVERSIFICADO,OFICIAL,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),SAN MARCOS
11094,12-16-1237-46,NaN,SAN MARCOS,CATARINA,EPRM,CASERIO LA INDEPENDENCIA,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,RURAL,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),SAN MARCOS
11120,12-17-1151-46,NaN,SAN MARCOS,AYUTLA,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,1A. CALLE Y 1A. AVENIDA ZONA 1,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,DOBLE,FIN DE SEMANA,SAN MARCOS


In [95]:
display(df.head(10))


,CODIGO,DISTRITO,Departamento,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,02-01-0027-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO TECNICO INDUSTRIAL MIXTO GUASTATOYA,BARRIO EL CALVARIO,79450881,CARLA MARLENY ALDANA RODAS,JOSɠARTURO LӐEZ ORTIZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),EL PROGRESO
1,02-01-0028-46,02-014,EL PROGRESO,GUASTATOYA,COLEGIO DE CIENCIAS COMERCIALES EL PROGRESO,BARRIO EL PORVENIR,79451265,CARLA MARLENY ALDANA RODAS,DANIEL LӐEZ SOL͓,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),EL PROGRESO
2,02-01-0031-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,BARRIO EL PORVENIR,54422753,CARLA MARLENY ALDANA RODAS,ILIANA ARACELY LZARO HERNNDEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
3,02-01-0038-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO DE EDUCACION MEDIA POR MADUREZ,"COLONIA NUEVA VIDA, BARRIO EL PORVENIR",7945177,CARLA MARLENY ALDANA RODAS,NORA REBECA IBAхZ MORN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO
4,02-01-0044-46,02-004,EL PROGRESO,GUASTATOYA,INSTITUTO OFICIAL DE EDUCACION BASICA Y BACHIL...,BARRIO LAS JOYAS,56962830,EDGAR RUDY MORALES GODINEZ,MARIA ELENA LEMUS MURALLES DE AREVALO,DIVERSIFICADO,OFICIAL,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
5,02-01-0045-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO DE EDUCACIӎ MEDIA POR MADUREZ,BARRIO EL PORVENIR,56719955,CARLA MARLENY ALDANA RODAS,NORA REBECA IBAхZ MORN,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO
6,02-01-0046-46,02-014,EL PROGRESO,GUASTATOYA,COLEGIO DE APLICACIONES INTEGRADAS GUASTATOYA,BARRIO EL PORVENIR,79450840,CARLA MARLENY ALDANA RODAS,BRENDA AZUCENA VALIENTE ARREAGA,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,SABATINO,EL PROGRESO
7,02-01-0049-46,02-021,EL PROGRESO,GUASTATOYA,CENTRO MUNICIPAL DE EDUCACIӎ EXTRAESCOLAR -CEEX-,COLONIA LINDA VISTA,58250827,MARTA ELIDA CARIAS HERNANDEZ DE GARCIA,SANDRA PAOLA MORALES GARĆ,DIVERSIFICADO,MUNICIPAL,RURAL,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,EL PROGRESO
8,02-01-0054-46,02-012,EL PROGRESO,GUASTATOYA,COLEGIO EVANGELICO TORRE FUERTE,"COLONIA HICHOS, BARRIO EL PORVENIR",79451993,CARLOTA EUGENIA ALBUREZ AGUILAR,IRIS ORFEL͠LOAIZA MOSCOSO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO
9,02-01-0057-46,02-014,EL PROGRESO,GUASTATOYA,COLEGIO HIGA,COLONIA HICHOS,53947190,CARLA MARLENY ALDANA RODAS,ERIKA JULISA OLIVA DUARTE,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),EL PROGRESO


In [103]:
# Valores únicos (sin repeticiones)
valores_unicos = df["ESTABLECIMIENTO"].unique()

# Mostrar resultados
print("Valores únicos en la columna:")
print(valores_unicos)

Valores únicos en la columna:
['INSTITUTO TECNICO INDUSTRIAL MIXTO GUASTATOYA'
 'COLEGIO DE CIENCIAS COMERCIALES EL PROGRESO'
 'INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA' ...
 "'COLEGIO EDUCATIVO MIXTO LUNA AZUL -CEMLA-'"
 "CENTRO EDUCATIVO 'CRISTO EL SALVADOR' FE Y ALEGŔ NO. 03"
 'COLEGIO EDUCATIVO MIXTO LUNA AZUL -CEMLA-']
